In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pickle
import sys
import numpy as np

np.set_printoptions(precision=3, suppress=True)
# uncomment and replace with correct path if there are import errors
# sys.path.append("/path/to/surface-sampling/")
# sys.path.append("/path/to/NeuralForceField/")
# os.environ["LAMMPS_POTENTIALS"] = "/path/to/lammps/potentials/"
# os.environ["LAMMPS_COMMAND"] ="/path/to/lammps/src/lmp_serial"
# os.environ["ASE_LAMMPSRUN_COMMAND"] = os.environ["LAMMPS_COMMAND"]

from mcmc import MCMC
from mcmc.system import SurfaceSystem

from time import perf_counter


Initialize test slab and parameters

In [3]:
# Load prepared pristine slab
element = "Ga"
slab_pkl = open("../tutorials/data/GaN_0001_3x3_pristine_slab.pkl", "rb")
slab = pickle.load(slab_pkl)

system_settings = {
    "surface_name": 'GaN(0001)',
    "cutoff": 5.0,
    "num_ads_atoms": 12,
    "near_reduce": 0.01,
    "planar_distance": 1.5,
    "no_obtuse_hollow": True,
}

sampling_settings = {
    "alpha": 0.99, # slowly anneal
    "temperature": 1.0, # in terms of kbT
    "num_sweeps": 50,
    "sweep_size": 104,
}

calc_settings = {
    "calc_name": "LAMMPS",
    "optimizer": "LAMMPS",
    "chem_pots": {"Ga": 5}, # eV. arbitrary value
    "relax_atoms": True,
    "relax_steps": 100,
    "run_dir": "/home/dux/surface_sampling/tests/resources/GaN_0001",
}


Obtain adsorption sites

In [4]:
from pymatgen.analysis.adsorption import AdsorbateSiteFinder
from pymatgen.io.ase import AseAtomsAdaptor

pristine_slab = slab.copy()
pristine_pmg_slab = AseAtomsAdaptor.get_structure(pristine_slab)
site_finder = AdsorbateSiteFinder(pristine_pmg_slab)
sites = site_finder.find_adsorption_sites(put_inside=True, symm_reduce=False)

ads_positions = sites['all']
print(f"adsorption coordinates are: {ads_positions[:5]}...")

adsorption coordinates are: [array([ 0.   ,  0.   , 15.266]), array([12.865,  7.428, 14.62 ]), array([12.865,  5.571, 15.266]), array([11.257,  4.642, 14.62 ]), array([ 1.608,  2.785, 15.266])]...


In [5]:
pristine_slab.get_tags()

array([4, 3, 2, 1, 4, 3, 2, 1, 4, 3, 2, 1, 4, 3, 2, 1, 4, 3, 2, 1, 4, 3,
       2, 1, 4, 3, 2, 1, 4, 3, 2, 1, 4, 3, 2, 1])

Set up LAMMPS (calculator). Requires a `lammps_config.json` and a `lammps_template.txt` in the same directory

In [6]:
from mcmc.calculators import LAMMMPSCalc, LAMMPSSurfCalc

lammps_surf_calc = LAMMPSSurfCalc()
lammps_surf_calc.set(**calc_settings)

# ! ln -sf data/GaN_lammps_config.json lammps_config.json
# ! ln -sf data/GaN_lammps_energy_template.txt lammps_energy_template.txt
# ! ln -sf data/GaN_lammps_opt_template.txt lammps_opt_template.txt

run directory: /home/dux/surface_sampling/tests/resources/GaN_0001 is set from parameters
relaxation steps: 100 is set from parameters


Initialize surface system

In [8]:
# fix bulk atoms
num_bulk_atoms = len(pristine_slab)
bulk_indices = list(range(num_bulk_atoms))
print(f"bulk indices {bulk_indices}")
surf_indices = pristine_slab.get_surface_atoms()

fix_indices = list(set(bulk_indices) - set(surf_indices))
print(f"fix indices {fix_indices}")

# bulk atoms are 0 and surface atoms are 1
tags = np.ones(num_bulk_atoms)
tags[fix_indices] = 0
pristine_slab.set_tags(tags)
print(f"tags {tags}")

surface = SurfaceSystem(pristine_slab, ads_positions, calc=lammps_surf_calc, system_settings=system_settings)
surface.all_atoms.write("GaN_0001_3x3_all_virtual_ads.cif")

2024-02-13 10:30:58,389|INFO|initial state is [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
2024-02-13 10:30:58,390|INFO|number of pristine atoms is 36
2024-02-13 10:30:58,391|INFO|bulk indices are [ 2  3  6  7 10 11 14 15 18 19 22 23 26 27 30 31 34 35]
2024-02-13 10:30:58,392|INFO|surface indices are [ 0  1  4  5  8  9 12 13 16 17 20 21 24 25 28 29 32 33]


bulk indices [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
fix indices [2, 3, 34, 35, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31]
tags [1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.
 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0.]
ads coords is [ 0.     0.    15.266]
ads coords is [12.865  7.428 14.62 ]
ads coords is [12.865  5.571 15.266]
ads coords is [11.257  4.642 14.62 ]
ads coords is [ 1.608  2.785 15.266]
ads coords is [ 9.649  1.857 14.62 ]
ads coords is [ 6.433  0.    15.266]
ads coords is [ 9.649  7.428 14.62 ]
ads coords is [ 9.649  5.571 15.266]
ads coords is [ 8.041  4.642 14.62 ]
ads coords is [ 8.041  2.785 15.266]
ads coords is [ 6.433  1.857 14.62 ]
ads coords is [ 3.216  0.    15.266]
ads coords is [ 6.433  7.428 14.62 ]
ads coords is [ 6.433  5.571 15.266]
ads coords is [ 4.824  4.642 14.62 ]
ads coords is [ 4.824  2.785 15.266]
ads coords is [ 3.216  1.8

Test calculate pristine surface

In [11]:
print(f"Energy {surface.get_potential_energy():.3f} eV")

Total wall time: 0:00:00
Energy -144.059 eV


Perform MCMC and view results. Detailed results can be found in the corresponding run in the `GaN(0001)/` folder.

In [14]:
mcmc = MCMC(
    system_settings['surface_name'], 
    calc=lammps_surf_calc, 
    canonical=True,
    testing=False,
    element=element,
    adsorbates=list(calc_settings['chem_pots'].keys()), 
    relax=calc_settings['relax_atoms'], 
    relax_steps=calc_settings['relax_steps'],
    optimizer=calc_settings['optimizer']),
    num_ads_atoms=system_settings['num_ads_atoms']
)

start = perf_counter()
# TODO: convert input to sampling settings
mcmc.mcmc_run(
    total_sweeps=sampling_settings['num_sweeps'],
    sweep_size=sampling_settings['sweep_size'],
    start_temp=sampling_settings['temperature'],
    pot=list(calc_settings['chem_pots'].values()),
    alpha=sampling_settings['alpha'],
    surface=surface
    )
stop = perf_counter()
print(f"Time taken = {stop - start} seconds")

SyntaxError: cannot assign to function call (102788686.py, line 2)